<div dir="rtl">
<h3>ترکیب داده‌های جغرافیایی مختلف برای به دست آوردن جزییات</h3>
</div>

<br />

<a href="https://www.youtube.com/watch?v=UM4S5yUsexo&list=PLY_JJbHwd6kftjlcbtChIKZfEPEfFRGcY&index=22&ab_channel=TheInternationalJournalists%27Network">
<img src="https://dotnet-talk.com/images/icfj/2_5.jpg" width="50%" alt="ترکیب دادهای جغرافیایی مختلف برای بدس آوردن جزییات - ویدیو" />
</a>

<br />

<div dir="rtl">
<h4>تنظیمات</h4>
</div>

<div dir="rtl">
ما قصد داریم از یک کتابخانه جدید به نام <a href="https://geopandas.org/en/stable/">geopandas</a> استفاده کنیم تا مجموعه داده‌ها با اطلاعات جغرافیایی را بخوانیم.
</div>

In [ ]:
!pip3 install folium==0.14.0 matplotlib==3.7.1 mapclassify==2.5.0

<div dir="rtl">
<p>
ما کتابخانه‌های folium و matplotlib را وارد می‌کنیم (کتابخانه‌هایی که در درس‌های قبلی دیده‌ایم) زیرا می‌توانیم از این کتابخانه‌ها برای تجسم چندین نوع داده جغرافیایی (مانند چندضلعی‌ها و نقاط) استفاده کنیم.
</p>
</div>

In [ ]:
# import necessary libraries

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium

<div dir="rtl">
<h2>بارگیری مجموعه‌داده‌ها</h2>


<p>ما داده‌های دو پورتال اطلاعات باز ایران را تجسم می‌کنیم. مجموعه‌داده اول یک مجموعه‌داده جغرافیایی دربارهٔ <a href="https://raw.githubusercontent.com/kwonjs/datasets-to-use/main/iranopendata-university-dataset.geojson">موسسات آموزش عالی</a> است. مجموعه‌داده دوم دربارهٔ <a href="https://raw.githubusercontent.com/kwonjs/datasets-to-use/main/sciehe1735en.csv">اشتغال/داده‌های اقتصادی</a> در ایران است. می‌توانید با کلیک بر روی پیوندها آن‌ها را <strong>مستقیماً بارگیری</strong> کنید.</p>

<p>شما همچنین می‌توانید با کلیک بر روی پیوندها، درباره متن کامل و راهنمایی‌های بیشتر درباره <a href="https://iranopendata.org/en/dataset/literate-employed-population-qualification-terms-gender-province-1394">اشتغال</a> و <a href="https://irandataportal.syr.edu/colleges-and-universities">آموزش</a> بیشتر بدانید.</p>

</div>


<div dir="rtl">
<p>حال مجموعه داده‌های اولیه را در نوت‌بوک Jupyter بارگیری می کنیم!</p>
</div>

In [ ]:
# importing the datasets as datasets

# geojson file of higher education institutes (colleges and universities) in iran
# non-comprehensive list from iran open data (syracuse edu)
univ_file_path = "../datasets/geo/iranopendata-university-dataset.geojson"
universities = gpd.read_file(univ_file_path)

# geojson file of iran's provinces
province_file_path = "../datasets/geo/iran-provinces.geojson"
provinces = gpd.read_file(province_file_path)

# csv of employment and economic participation by province
employment_file_path = "../datasets/csv/sciehe1735en.csv"
employment = pd.read_csv(employment_file_path)


<div dir="rtl">
<p>سپس نگاه سریعی به مجموعه داده‌هایی که با آن‌ها کار خواهیم کرد می اندازیم.</p>
</div>

In [ ]:
# seeing the first five rows of the employment data
employment.head(5)

In [ ]:
# seeing the first five rows of the university/college data
universities.head(5)

In [ ]:
# seeing the first five rows of the provinces data
provinces.head(5)

<div dir="rtl">
<em>توجه: مجموعه داده دانشگاه و کالج شامل داده‌های نقطه‌ای است در حالی که مجموعه داده استان + اشتغال حاوی داده‌های چندضلعی / مرزی است. مجموعه داده دانشگاه / کالج جامع نیست اما برای هدف این درس کاربرد دارد.</em>
</div>

<div dir="rtl">
<h2>تمیزکاری و آماده‌سازی داده برای نمایش</h2>

<p>
دیتاست اشتغال بر اساس جنسیت و <strong>نام استان</strong> سازمان‌دهی شده است. می‌توانیم داده‌های geojson (شامل هر استان و مختصات جغرافیایی مرزهای آن) را با داده‌های اشتغال با استفاده از <strong>نام استان</strong> (در ستون "english_name") ارتباط بدهیم.
</p>

<p>
برای اتصال این دو مجموعه داده، ابتدا باید مطمئن شویم که نام استان‌ها در ستون‌های هر دو مجموعه داده <strong>یکسان</strong> باشد تا بتوانیم این دو مجموعه داده را به هم وصل کنیم. یکی از مجموعه داده شامل داده‌های واقعی چندضلعی برای هر استان است که برای نمایش روی نقشه لازم است و دیگری حاوی داده‌های اشتغال مرتبط با هر استان است.
</p>

</div>

<div dir="rtl">
ابتدا بیایید ستون "جنسیت و استان" را در دیتافریم اشتغال به دو ستون تقسیم کنیم، یکی برای جنسیت و دیگری برای استان. سپس می‌توانیم ستون استان را جدا کنیم تا بتوانیم آن را با ستون <strong>"english_name"</strong> (یعنی نام استان به زبان انگلیسی) در دیتافریم استان‌ها مطابقت دهیم.
الگوی کلی ستون "جنسیت و استان" به صورت <strong>[نام استان] (جنسیت)</strong> به نظر می‌رسد. ابتدا بیایید این ستون را با پرانتز چپ، یعنی '('، تقسیم کنیم.

</div>

In [ ]:
# Splitting up the "Gender and province" column by the left parenthesis
gender_province_cols1 = employment["Gender and province"].str.split('(', expand=True)
gender_province_cols1.head()

<div dir="rtl">
<p>
ستون '0' (ستون اول) تقریباً تمامی استان‌ها را شامل می‌شود (به استثنای ستون اول که "مرد و زن" را دارد، اما می‌توانیم این مورد را بعداً تصحیح کنیم). بیایید این ستون را جدا کنیم و آن را 'isolated_province_col' نامگذاری کنیم.
</p>
</div>

In [ ]:
isolated_provinces_col = gender_province_cols1[0] # the '0' index selects for the first column in this separate dataframe
isolated_provinces_col

<div dir="rtl">
<p>
ما ستون اصلی را جدا کردیم، اما جداکردن کامل نبود. هنوز یک پرانتز راست بی‌استفاده در ستون دوم به وجود آمده است (که باید حاوی جنسیت باشد). بیایید این ستون را هم جدا کنیم تا از آن پرانتز راست خلاص شویم.
</p>
</div>

In [ ]:
# Splitting up the newly made 'gender' column (or the second column) by the right parenthesis as well
# Then assign the first column to 'isolated_gender_col,' since we've now successfully gotten a mostly clean gender column
isolated_gender_col = gender_province_cols1[1].str.split(')', expand=True)[0]
isolated_gender_col

<div dir="rtl">
حالا این ستون‌های جدا شده و تقریباً پاک را به دیتافریم اشتغال اضافه می کنیم.
</div>

In [ ]:
employment["province"] = isolated_provinces_col # adds this column to the employment dataset and calls this column "Province"
employment["gender"] = isolated_gender_col # adds this column to the employment dataset and calls this column "Gender"

# Let's load in the first five rows of the employment dataframe to check that we successfully added these new columns into the dataframe
#employment.head(5) # to check

<div dir="rtl">
<p>
حالا باید ستون "استان" ایجاد شده در دیتافریم employment  را با ستون "english_name" در دیتافریم استان‌ها مطابقت دهیم.
</p>
<p>
بیایید نام‌های استان موجود در ستون "english_name" و "Province" را مقایسه کنیم.
</p>
</div>


In [ ]:
# Get unique values in "english_name"
# Get unique values in "Province"
# If they don't match - change a value in one column to match the other
provinces["english_name"].unique()
# employment[["Province", "Gender"]]

In [ ]:
employment["province"].unique()

In [ ]:
differences = list(set(provinces["english_name"]).difference(employment["province"]))
differences

<div dir="rtl">
بیایید نام‌های استان متفاوت را در مجموعه داده اشتغال تغییر نام دهیم. (بسیاری از این نام‌ها فقط با یک 'space' متفاوت هستند که هنگام تقسیم ستون "Gender and province" باقی مانده است.)
</div>

In [ ]:
employment = employment.replace({'province' : {
    'Mazandaran ' : 'Mazandaran',
    'Khorasan Razavi ' : 'Razavi Khorasan',
    'Kermanshah ' : 'Kermanshah',
    'Yazd ' : 'Yazd',
    'Isfahan ' : 'Isfahan',
    'Kurdistan ' : 'Kurdistan',
    'Gilan ' : 'Gilan',
    'Khuzestan ' : 'Khuzestan',
    'Khorasan South ' : 'South Khorasan',
    'Markazi ' : 'Markazi',
    'Sistan and Baluchestan ' : 'Sistan and Baluchestan',
    'Golestan ' : 'Golestan',
    'Ilam ' : 'Ilam',
    'Bushehr ' : 'Bushehr',
    'Qazvin ' : 'Qazvin',
    'Kohgiluyeh and Boyer-Ahmad ' : 'Kohgiluyeh and Boyer Ahmad',
    'Zanjan ' : 'Zanjan',
    'Tehran ' : 'Tehran',
    'Qom ' : 'Qom',
    'Lorestan ' : 'Lorestan',
    'Fars ' : 'Fars',
    'Kerman ' : 'Kerman',
    'Khorasan North ' : 'North Khorasan',
    'Azerbaijan East ' : 'East Azarbaijan',
    'Ardabil ' : 'Ardabil',
    'Hamadan ' : 'Hamadan',
    'Hormozgan ' : 'Hormozgan',
    'Azerbaijan West ' : 'West Azarbaijan',
    'Alborz ' : 'Alborz',
    'Semnan ' : 'Semnan',
    'Chahar Mahaal and Bakhtiari ' : 'Chaharmahal and Bakhtiari'
    }})

<div dir="rtl">
<p>
حال که نام استان‌ها در هر دو مجموعه داده استخدام و استان یکسان است، بیایید این دو مجموعه داده را با هم ترکیب کنیم.
</p>
<p>
ابتدا، بیایید نام ستون "english_name" را در مجموعه داده استخدام به همان نام ستون، "Province"، تغییر نام دهیم.
</p>
</div>

In [ ]:
provinces = provinces.rename(columns={'english_name': 'province'})

In [ ]:
employment_province = provinces.merge(employment, on='province', how='left')
employment_province

<div dir="rtl">
حدود جغرافیایی مرتبط با هر استان به داده‌های مرتبط با اشتغال نیز مرتبط است.
</div>

<div dir="rtl">
توجه کنید که انواع مختلفی از داده‌ها به همان استان مرتبط هستند - هر استان داده‌های مربوط به نرخ بیکاری و زیراشتغال برای جنسیت زن ("Female" در ستون "Gender" در دیتافریم employment_province)، جنسیت مرد ("Male") و جنسیت مرد و زن ("Male and Female") دارد. بیایید به طور جغرافیایی داده‌های "Male and Female" را نمایش دهیم.
</div>

In [ ]:
employment_province_mf = employment_province[(employment_province["gender"] == "Male and Female")]

<div dir="rtl">
حالا بیایید این داده‌های جغرافیایی فیلتر شده را با داده‌های مکان دانشگاه/کالج با استفاده از <a href="https://python-visualization.github.io/folium/modules.html#module-folium.map" dir="ltr">folium.Map()</a> نقشه پردازی کنیم.
</div>

In [ ]:
# Creating a map object in Folium
# Setting the latitude and longitude (in this case, the center of Iran)
m = folium.Map(location=[32.4279, 53.6880], zoom_start=5.25)
m

<div dir="rtl">
حالا بیایید فقط داده‌های استانی را فیلتر کنیم که شامل هر دو جنس مرد و زن باشند. به عبارت دیگر، ستون "جنسیت" باید فقط شامل داده‌های "مرد و زن" باشد.
</div>

In [ ]:
employment_mf = employment[employment["gender"] == "Male and Female"]

<div dir="rtl">
حالا نام ستون "english_name" را به همان نام ستون "استان" که در مجموعه داده‌های اشتغال وجود دارد، تغییر نام می دهیم. این کار به ما کمک می‌کند تا این دو مجموعه داده را به هم مرتبط کنیم، یکی شامل داده‌هایی برای رسم مرزهای استان‌ها و دیگری شامل داده‌های اشتغال برای نمایش در نقشه Folium.
</div>

In [ ]:
provinces = provinces.rename(columns={'english_name': 'province'})

<div dir="rtl">
حالا مجموعه داده‌های استان‌ها را به فرمت GeoJSON تبدیل می کنیم تا بتوانیم از آن با کتابخانه Folium استفاده کنیم.
</div>

In [ ]:
provinces_gdf = gpd.GeoDataFrame(provinces, geometry="geometry")
provinces_gdf.to_file("provinces.geojson", driver="GeoJSON")

provinces = gpd.read_file("provinces.geojson")

In [ ]:
employment_province_mf

<h2 dir="rtl">ترسیم نقشه با استفاده از geopandas</h2>
<div dir="rtl">
حالا بیایید اطلاعات موقعیت جغرافیایی استخدامی فیلتر شده را با اطلاعات موقعیت دانشگاه/کالج ها نقشه برداری کنیم.
</div>

In [ ]:
# Plotting a simple choropleth map with geopandas
choro = employment_province_mf.plot(column = 'Proportion of the population with underemployment (All country) - percent', # Data column to visualize
                            legend = True, # It's good practice to show a legend
                            figsize = [10,5], # Set chart dimensions
                            legend_kwds = {'label': "Percentage of population underemployed by province"}) # Adding a label title to describe what data you're visualizing

# The axes in this case represent latitude and longitude

<div dir="rtl">
حالا هم نقشه کروپلت (که با سایه نمایش می‌دهد که چه درصدی از جمعیت بیکار استخدام زیرشغلی دارند، براساس هر استان) را برسیم و <strong>همچنین</strong> نقاطی که مکان دانشگاه‌ها و مؤسسات آموزش عالی در سراسر ایران است را نمایش دهیم.
</div>

In [ ]:
fig, ax = plt.subplots(figsize=(7,7)) # Since we're visualizing more than one type of data

# First, we plot the choropleth map showing the underemployment percentages by province
# The lighter the shade of purple, the lower the underemployment percentage
employment_province_mf.plot(ax=ax,
                      column='Proportion of the population with underemployment (All country) - percent', # Data column to visualize
                      legend=True,
                      edgecolor='black',
                      linewidth=0.5,
                      cmap="Reds",
                      legend_kwds = {'label': "Percentage of population underemployed by province"})

# Then, on top of the choropleth map we just made, we plot the points showing where colleges and universities are
# Each black point represents one college or university
universities.plot(ax=ax,
                  color='Blue',
                  markersize=5)

ax.set_title('College locations and underemployment rates by province', fontsize=10)
ax.set_xlabel('Longitude', fontsize=8)
ax.set_ylabel('Latitude', fontsize=8)

<div dir="rtl">
ما تازه نقشه‌های استاتیک را ایجاد کردیم - اما می‌توانیم با استفاده از <strong style="color: orange" dir="ltr">geopandas.explore()</strong> برخی از تعاملیت را اضافه کنیم. این عمل بازگشت یک نقشه Folium را فراهم می‌کند که می‌توانید روی آن نمایش داده‌ها را ببینید و ببینید که در استانی قرار دارید و داده‌های مربوط به آن استان چیست.
</div>
<br />
<div dir="rtl">
مجموعه داده‌ی اشتغال دارای بسیاری از متغیرها/ستون‌ها است، بنابراین اضافه کردن پارامتر 'tooltip' در داخل <strong style="color: orange" dir="ltr">explore()</strong> به شما امکان می‌دهد تا به متغیرها/ستون‌های خاصی که می‌خواهید به آن تمرکز کنید، محدود شوید.
</div>

In [ ]:
import mapclassify as mp
employment_province_mf.explore(
    column='Proportion of the population with underemployment (All country) - percent',
    cmap='Purples',
    tooltip=['province', 'Proportion of the population with underemployment (All country) - percent'] # the specific columns to highlight in the tooltip
)

In [ ]:
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

# Now we can create a choropleth map with these two datasets

folium.Choropleth(geo_data = provinces, # geo_data refers to your geojson file (with data used to draw geographic boundaries)
    name = "choropleth",
    data = employment_mf, # data refers to the non-geographic data you're trying to tie to geographic data
    columns = ["province", "Employment rate (All country) - percent"], # what data are you wanting to visualize?
    key_on = "feature.properties.province", # what column ties both datasets? in this case, it's the name of the province, or the "Province" column (this is why it's important that both columns have the same name)
    fill_color = 'YlOrRd',
    fill_opacity = 0.3,
    line_opacity = 0.9,
    legend_name = 'Employment rate (percent)').add_to(m)

folium.LayerControl().add_to(m)
m # call the the map again after adding this data!

<div dir="rtl">
این بهترین نقشه‌ی کروپلت نیست - همانطور که می‌بینید، بیشتر استان‌ها با رنگ قرمز مشابه رنگ شده‌اند، زیرا نرخ اشتغال در اکثر استان‌ها در محدودهٔ ۸۰ درصد است که در محدودهٔ بالای نموداری که در بالای نقشه نشان داده شده است قرار می‌گیرد. این ستون خاص که ما نمایش می‌دهیم، "نرخ اشتغال (کل کشور) - درصد"، شاید بهترین متغیر برای تصویرسازی با استفاده از نقشهٔ کروپلت نباشد. ممکن است ما یک متغیر دیگر با داده‌های پراکنده در یک محدوده انتخاب کنیم، مانند نقشه‌ای که با استفاده از <strong style="color: orange" dir="ltr">explore()</strong> ایجاد کرده بودیم برای تصویرسازی نسبت جمعیت هر استان که تحت بیکاری قرار می‌گیرد.
</div>

<h2 dir="rtl">
منابع برای نقشه نگاری با پایتون
</h2>
<br />
<div dir="rtl">
ما مطالب زیادی را در این درس یاد گرفتیم! شما می‌توانید با نقشه‌های کلانشهری و نقشه‌های نقطه‌ای تنوع زیادی از آنچه که در اینجا انجام دادیم، انجام دهید. در ادامه، منابع بیشتری برای کاوش عمیق‌تر در این حوزه برای شما قرار داده شده است.
</div>

<ol dir="rtl">
<li>
<a href="https://medium.com/geekculture/three-ways-to-plot-choropleth-map-using-python-f53799a3e623">سه راه برای ساختن نقشه در پایتون</a>
</li>
<li>
<a href="https://matplotlib.org/stable/tutorials/colors/colormaps.html">انتخاب پالت رنگ </a> برای نقشه های کروپلت
</li>
<li>
<a href="https://geopandas.org/en/stable/gallery/create_geopandas_from_pandas.html">ایجاد geopandas از pandas</a>
</li>
<li>
<a href="https://towardsdatascience.com/plotting-maps-with-geopandas-428c97295a73">رسم نقشه‌ها با استفاده از geopandas</a>
</li>
</ol>